In [1]:
from kinlpmorpho import build_kinlp_morpho_lib

build_kinlp_morpho_lib()

from morpho_data_loaders import KBVocab, ParsedToken, AffixSetVocab,  read_corpus

import torch

print('Imports OK!')

generating ./kinlpmorpholib.c
(already up-to-date)
the current directory is '/mnt/NVM/projects/user/kinyabert/modeling/kinyabert'
running build_ext
building 'kinlpmorpholib' extension
gcc -pthread -B /home/user/anaconda3/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/user/anaconda3/include/python3.8 -c kinlpmorpholib.c -o ./kinlpmorpholib.o -fopenmp -D use_openmp -O3 -march=native -ffast-math
gcc -pthread -shared -B /home/user/anaconda3/compiler_compat -L/home/user/anaconda3/lib -Wl,-rpath=/home/user/anaconda3/lib -Wl,--no-as-needed -Wl,--sysroot=/ ./kinlpmorpholib.o -lkinlp -o ./kinlpmorpholib.cpython-38-x86_64-linux-gnu.so -fopenmp
Imports OK!


In [2]:
# corpus_lines = read_corpus('/mnt/NVM/KINLP/data/parsed_tfidf_corpus_2021-02-07.txt')

print('Corpus Read OK!')

Corpus Read OK!


In [3]:
import torch
kb_vocab = KBVocab()
kbvocab_state_dict_file_path = ("/mnt/NVM/KINLP/data/kb_vocab_state_dict_2021-02-07.pt")
kb_vocab.load_state_dict(torch.load(kbvocab_state_dict_file_path))

print('Stems: ', len(kb_vocab.reduced_stem_vocab))

Stems:  34008


In [4]:
# affix_dict = dict()
# affix_txt_dict = dict()
# for id,line in enumerate(corpus_lines):
#     splits = line.split('; ')
#     if (len(splits) > 0):
#         if ((len(splits[0]) > 4) and ('/' in splits[0]) and (':' in splits[0])):
#             parsed_tokens_line = [ParsedToken('_', parsed_token=t, line_num=id) for t in splits]
#             for pt in parsed_tokens_line:
#                 key = '-'.join([str(af) for af in pt.affixes_idx]) if (len(pt.affixes_idx)>0) else 'N/A'
#                 txt_key = '-'.join([kb_vocab.affix_vocab_idx[af] for af in pt.affixes_idx]) if (len(pt.affixes_idx)>0) else 'N/A'
#                 val = 0
#                 if key in affix_dict:
#                     val = affix_dict[key]
#                 affix_dict[key] = val + 1
#                 affix_txt_dict[txt_key] = val + 1
#
# sorted_affix_dict = {k: v for k, v in sorted(affix_dict.items(), key=lambda item: item[1])}
# sorted_affix_txt_dict = {k: v for k, v in sorted(affix_txt_dict.items(), key=lambda item: item[1])}
#
# del corpus_lines

print('Data counting OK!')

Data counting OK!


In [9]:
# sorted_affix_dict = {k: v for k, v in sorted(affix_dict.items(), reverse=True, key=lambda item: item[1])}
# sorted_affix_txt_dict = {('-'.join([kb_vocab.affix_vocab_idx[int(x)] if (k!='N/A') else k for x in k.split('-')])): v for k, v in sorted(affix_dict.items(), reverse=True, key=lambda item: item[1])}

In [5]:
# f = open("/home/user/Desktop/Desk/sorted_affix_dict.csv", "w")
# for k in sorted_affix_dict:
#     f.write(k+","+str(sorted_affix_dict[k])+"\n")
# f.close()
#
# f = open("/home/user/Desktop/Desk/sorted_affix_txt_dict.csv", "w")
# for k in sorted_affix_txt_dict:
#     f.write(k+","+str(sorted_affix_txt_dict[k])+"\n")
# f.close()

print('Output files OK!')

Output files OK!


In [4]:
import progressbar

def reduce_affix_set_vocab(sorted_affix_dict_obj, N):
    sorted_affix_list = [(x,sorted_affix_dict_obj[x]) for x in sorted_affix_dict_obj]
    reduced_affix_list = sorted_affix_list[:N]
    # Make least frequent afsets better candidates for dictionary reduction
    reduced_affix_list.reverse()
    discarded_affix_list = sorted_affix_list[N:]
    reduced_affix_dict = {k:v for (k,v) in reduced_affix_list}
    reduced_affix_dict_map = {k:k for (k,v) in reduced_affix_list}

    reduced_affix_sets = {k:set([int(i) for i in k.split('-') if k!='N/A']) for (k,v) in reduced_affix_list}
    discarded_affix_sets = {k:set([int(i) for i in k.split('-') if k!='N/A']) for (k,v) in discarded_affix_list}

    idx = 0
    with progressbar.ProgressBar(initial_value=0, max_value=len(discarded_affix_sets), redirect_stdout=True) as bar:
        for k in discarded_affix_sets:
            bar.update(idx)
            idx += 1
            s = discarded_affix_sets[k]
            len_vals = [num for num in range(1,len(s))]
            len_vals.reverse()
            if len_vals is None:
                len_vals = []
            for n in len_vals:
                for kk in reduced_affix_sets:
                    ss = reduced_affix_sets[kk]
                    if (len(ss)==n):
                        if ss.issubset(s):
                            reduced_affix_dict_map[k]=kk
                            break
                if k in reduced_affix_dict_map:
                    break
            if not k in reduced_affix_dict_map:
                reduced_affix_dict_map[k] = 'N/A'
            reduced_affix_dict[reduced_affix_dict_map[k]] = reduced_affix_dict[reduced_affix_dict_map[k]] + sorted_affix_dict_obj[k]
    return reduced_affix_dict, reduced_affix_dict_map


In [5]:
f = open('/home/user/Desktop/Desk/sorted_affix_dict.csv', 'r+')
dict_lines = [line.rstrip('\n') for line in f]
f.close()
sorted_affix_dict = {(l.split(',')[0]):int(l.split(',')[1]) for l in dict_lines if len(l)>0}
print(len(sorted_affix_dict))

218468


In [6]:
MAX_NUM_SETS = len(kb_vocab.reduced_stem_vocab) #30000
print('MAX_NUM_SETS:', MAX_NUM_SETS)

reduced_affix_dict, reduced_affix_dict_map = reduce_affix_set_vocab(sorted_affix_dict, MAX_NUM_SETS)

print('Reduction OK!')

MAX_NUM_SETS: 34008


100% (184460 of 184460) |################| Elapsed Time: 0:11:03 Time:  0:11:03


Reduction OK!


In [11]:
print(len(reduced_affix_dict))
print(len(reduced_affix_dict_map))

34008
218468


In [12]:
reduced_affix_txt_dict = {('-'.join([kb_vocab.affix_vocab_idx[int(x)] if (k!='N/A') else k for x in k.split('-')])): reduced_affix_dict[k] for k in reduced_affix_dict}

reduced_affix_txt_dict_map = {('-'.join([kb_vocab.affix_vocab_idx[int(x)] if (k!='N/A') else k for x in k.split('-')])) : ('-'.join([kb_vocab.affix_vocab_idx[int(x)] if (reduced_affix_dict_map[k]!='N/A') else reduced_affix_dict_map[k] for x in reduced_affix_dict_map[k].split('-')])) for k in reduced_affix_dict_map}

In [13]:
f = open("/home/user/Desktop/Desk/reduced_affix_dict_"+str(MAX_NUM_SETS)+".csv", "w")

reverse_list = [k for k in reduced_affix_dict]
reverse_list.reverse()
for k in reverse_list:
    f.write(k+","+str(reduced_affix_dict[k])+"\n")
f.close()

f = open("/home/user/Desktop/Desk/reduced_affix_dict_map_"+str(MAX_NUM_SETS)+".csv", "w")

for k in reduced_affix_dict_map:
    f.write(k+","+reduced_affix_dict_map[k]+"\n")
f.close()

f = open("/home/user/Desktop/Desk/reduced_affix_txt_dict_"+str(MAX_NUM_SETS)+".csv", "w")
reverse_list = [k for k in reduced_affix_txt_dict]
reverse_list.reverse()
for k in reverse_list:
    f.write(k+","+str(reduced_affix_txt_dict[k])+"\n")
f.close()

f = open("/home/user/Desktop/Desk/reduced_affix_txt_dict_map_"+str(MAX_NUM_SETS)+".csv", "w")
for k in reduced_affix_txt_dict_map:
    f.write(k+","+reduced_affix_txt_dict_map[k]+"\n")
f.close()

In [10]:
print('Reading affix_set_vocab ...')

affix_set_vocab = AffixSetVocab(reduced_affix_dict_file='/mnt/NVM/KINLP/data/reduced_affix_dict_'+str(MAX_NUM_SETS)+'.csv',
              reduced_affix_dict_map_file='/mnt/NVM/KINLP/data/reduced_affix_dict_map_'+str(MAX_NUM_SETS)+'.csv')

print('affix_set_vocab read OK!')

# torch.save(affix_set_vocab.state_dict(),'/mnt/NVM/KINLP/data/affix_set_vocab_2021-08-13.pt')
#
# print('affix_set_vocab save OK!')

Reading affix_set_vocab ...
affix_set_vocab read OK!
